# Set-up 

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import sys 
sys.path.append('./src')
import pandas as pd 
import numpy as np
import re
import os
from tqdm import tqdm
from typing import NoReturn
import wonderwords
import random


In [8]:
DATA_DIR = '/home/prichter/Documents/data/methanotrophy'

In [4]:

def df_from_metadata(path:str) -> pd.DataFrame:
    '''Load the sample metadata.'''
    metadata_df = pd.read_csv(path)
    metadata_df.columns = [col.lower() for col in metadata_df.columns]
    metadata_df = metadata_df[~metadata_df.serial_code.isin(['PCR blank', 'Extr blank', 'KML'])] # Drop the outliers. 
    metadata_df.serial_code = metadata_df.serial_code.apply(int) # Convert serial code to integers.
    metadata_df.soil_depth = metadata_df.soil_depth.str.lower()
    return metadata_df


def df_from_taxonomy(path:str) -> pd.DataFrame:
    '''Load the ASV taxonomy information. The taxonomy file contains non-prokaryotes (e.g. nematodes and Eukaryotes), which 
    we keep in for the same of completeness.'''
    taxonomy_df = pd.read_csv(path, delimiter='\t', index_col=0)
    # Handled the unclassified taxa (they are entered as {some category}_unclassified)
    taxonomy_df = taxonomy_df.map(lambda s : s.replace('unclassified_', ''))

    for col in taxonomy_df.columns:
        # TODO: Might need a more general way to handle the Eukaryotes. 

        # Some of the nematode taxonomy labels are slightly irregular, as they contain clade information. The clades are discarded
        # for the same of simplicity (probably don't need much taxonomical resolution for the nematodes.)
        taxonomy_df[col] = taxonomy_df[col].apply(lambda s : 'Amorphea' if ('Amorphea' in s) else s)
        # SAR or Harosa is a highly diverse clade of eukaryotes, often considered a supergroup, that includes stramenopiles, alveolates,
        # and rhizarians (all Eukaryotes). Also make an exception for taxonomy labels of this kind. 
        taxonomy_df[col] = taxonomy_df[col].apply(lambda s : 'Harosa' if ('SAR' in s) else s)
        taxonomy_df[col] = taxonomy_df[col].apply(lambda s : 'Archaeplastida' if ('Archaeplastida' in s) else s)

        p = '([a-zA-Z0-9_]+)_([0-9]+)' # Pattern to match, all taxa ending in _{number}{number}
        # Create a new "sub" column for more taxonomical resolution if a numerical sub-category is given. 
        taxonomy_df[col + '_sub'] = taxonomy_df[col].apply(lambda s : int(re.match(p, s).group(2)) if not (re.match(p, s) is None) else 0)
        taxonomy_df[col] = taxonomy_df[col].apply(lambda s : re.match(p, s).group(1) if not (re.match(p, s) is None) else s)
    
    taxonomy_df.index.name = 'asv'
    return taxonomy_df


def df_from_counts(path:str) -> pd.DataFrame:
    '''Load the ASV count information.'''
    counts_df = pd.read_csv(path, delimiter='\t', index_col=0)
    counts_df['asv'] = counts_df.index
    # Convert columns to a categorical 'sample' variable. 
    counts_df = counts_df.melt(value_name='count', id_vars='asv', var_name='sample')
    # Throw out the weird samples.
    counts_df = counts_df[~counts_df['sample'].isin(['HDK-DNAexNegLot169030916-30cyc', 'HDK-MAR-PCR-BLANK'])]
    return counts_df.set_index('asv')

In [5]:
counts_df = df_from_counts(os.path.join(DATA_DIR, 'counts.tsv'))
taxonomy_df = df_from_taxonomy(os.path.join(DATA_DIR, 'taxonomy.tsv'))
metadata_df = df_from_metadata(f'{DATA_DIR}/metadata.csv')

missing_samples = set(counts_df['sample'].values) - set(metadata_df['sample'].values)
print(f"Samples {', '.join(list(missing_samples))} are present in the counts.tsv file, but not in metadata.")
print(f"Dropping {counts_df['sample'].isin(missing_samples).values.sum()} rows in the counts data which do not have associated metadata." )
counts_df = counts_df[~counts_df['sample'].isin(missing_samples)] # Remove rows in counts_df which don't have associated metadata.

sample_to_serial_code_map = {row.sample:row.serial_code for row in metadata_df.itertuples()}
counts_df['serial_code'] = counts_df['sample'].apply(lambda sample : sample_to_serial_code_map[sample])
counts_df = counts_df.drop(columns='sample')

# Combine the data across files into a single DataFrame. Drop the samples column, which is redundant with the serial code. 
df = counts_df.merge(taxonomy_df, how='left', left_index=True, right_index=True)

# Save the cleaned-up data to a CSV file. 
df.to_csv(f'{DATA_DIR}/data.csv') # 'asv' is the index, so make sure to include when writing. 
metadata_df.to_csv(f'{DATA_DIR}/metadata.csv', index=False)

Samples HDK22-KML-sand-dry, HDK22-KML-sand-wet are present in the counts.tsv file, but not in metadata.
Dropping 192136 rows in the counts data which do not have associated metadata.


# Artificial data

For testing the functionality of functions included in this software package, it is also necessary to generate some psuedo-count data, as well as artificial metadata. The artificial taxa counts are sampled from a normal distribution. Random taxa names for each taxonomy level (domain, kingdom, phylum, class, order, family, genus, species) are also generated. Metadata for each sample are also randomly generated, containing both categorical and continuous fake environmental variables. 

In [12]:
def create_artificial_taxonomy(n_asvs:int, n:int=3):
    '''Generate an artificial hierarchical taxonomy for n_asvs ASVs. The input n
    specifies the number of branches from each node in the artificial hierarchy.'''
    r = wonderwords.RandomWord() # Instantiate the random word generator. 

    levels = ['domain', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    df = pd.DataFrame(columns=['asv'] + levels) # Instantiate a DataFrame with the necessary columns.     
    df['asv'] = [f'ASV_{i}' for i in range(n_asvs)] # Add the ASVs to the DataFrame. 

    # Assign domains to each ASV to initialize the hierarchy. 
    domains = r.random_words(2, include_parts_of_speech=['noun'])
    df['domain'] = [domains[0].capitalize()] * (n_asvs // 2) + [domains[1].capitalize()] * (n_asvs - n_asvs // 2)

    for i in range(1, len(levels)): # Iterate over the phylogenetic levels in order of highest to lowest (skipping domain).
        prev_level, curr_level = levels[i - 1], levels[i]
        # Iterate over each name in the previous taxonomic level. 
        for t in set(df[prev_level].values):
            names = r.random_words(n, regex='\w+', include_parts_of_speech=['noun']) # Generate a list of n names for lower-level taxonomical category. 
            if curr_level != 'species': # For levels other than species, capitalize the name. 
                names = [name.capitalize() for name in names]
            n_t = df[prev_level].str.match(t).sum() # Get the number of ASVs matching the taxon in the previous level.
            df.loc[df[prev_level].str.match(t), curr_level] = [random.choice(names) for _ in range(n_t)]
    return df

In [18]:
def create_artificial_counts(n_asvs:int, n_samples:int, n_zeros:int=1000):
    '''Generate an artificial count matrix for n_asvs ASVs and n_samples samples. The count matrix is then converted to
    a melted pandas DataFrame to make merging with taxonomy data easier.'''
    
    # # Maximum number of zeros to ensure that at least every sample and every ASV has at least one nonzero entry. 
    # max_zeros = (n_samples * n_asvs) - max(n_asvs, n_samples) 
    # assert n_zeros < max_zeros, f'create_artificial_count_data: Cannot be more than {max_zeros} zeros in the count matrix.'

    # Requirement is to have at least one count in each ASV category and each sample. 
    counts = np.random.randint(low=1, high=5000, size=(n_samples, n_asvs), dtype=int)
    row_idxs, col_idxs = np.arange(n_samples), np.arange(n_asvs)
    counts[np.random.choice(row_idxs, size=n_zeros), np.random.choice(row_idxs, size=n_zeros)] = 0
    
    assert not np.any(counts.sum(axis=0) == 0), 'create_artificial_count_data: There are too many zeros! Try decreasing n_zeros and re-running.'
    assert not np.any(counts.sum(axis=1) == 0), 'create_artificial_count_data: There are too many zeros! Try decreasing n_zeros and re-running.'

    # Create a matrix with the count data. 
    df = pd.DataFrame(counts, index=np.arange(n_samples), columns=[f'ASV_{i}' for i in range(n_asvs)])
    # Make sure to set ignore_index=True so that the serial codes are preserved when melting. 
    df = df.melt(ignore_index=False, value_name='count', var_name='asv')
    df['serial_code'] = df.index # For some reason, if I don't set the serial code as a column, it gets dropped during the merge with taxonomy.

    return df 

In [21]:
def create_artificial_metadata(n_samples:int, n_categorical:int=1, n_continuous:int=1):
    '''Generate an artificial DataFrame of sample metadata with both continuous and categorical variables.'''
    max_num_categorical = 5
    max_val_continuous = 10

    r = wonderwords.RandomWord()
    categorical_vars = r.random_words(n_categorical, include_parts_of_speech=['adjective'])
    continuous_vars = r.random_words(n_continuous, include_parts_of_speech=['adjective'])

    df = pd.DataFrame(index=np.arange(n_samples), columns=categorical_vars + continuous_vars)
    for var in categorical_vars:
        df[var] = np.random.randint(low=1, high=max_num_categorical, size=n_samples) 
    for var in continuous_vars:
        df[var] = np.random.random(size=n_samples) * max_val_continuous
    df['serial_code'] = df.index
    return df

In [22]:
n_asvs = 1000
n_samples = 100

artificial_taxonomy_df = create_artificial_taxonomy(n_asvs)
artificial_counts_df = create_artificial_counts(n_asvs, n_samples)
artificial_metadata_df = create_artificial_metadata(n_samples)

artificial_counts_df.merge(artificial_taxonomy_df, on='asv', how='left').to_csv('/home/prichter/Documents/methanotrophy/tests/data.csv', index=False)
artificial_metadata_df.to_csv('/home/prichter/Documents/methanotrophy/tests/metadata.csv', index=False)